In [1]:
import pandas as pd
import json

from serlo.mysql import db



In [6]:
def get_keys(obj, stack):
    for k, v in obj.items():
      k2 = ([k] if k else []) + stack # don't return empty keys
      if v and isinstance(v, dict):
        for c in get_keys(v, k2):
          yield c
      else: # leaf
        yield k2

def get_values(obj):
    for v in obj.values():
      if not v: continue
      if isinstance(v, dict):
        for c in get_values(v):
          yield c
      else: # leaf
        yield v if isinstance(v, list) else [v]
    
        
def get_value_from_key_array(obj, keys):
    if len(keys) == 0:
        return
    if len(keys) == 1:
        return obj[keys[0]]
    else:
        key = keys.pop(-1)
        return get_value_from_key_array(obj[key], keys)

def leaf_equals(keys, wanted_key):
    return keys[0] == wanted_key

In [4]:
def is_plugin(state):
    if "plugin" in state:
        return True
    return False
    
def has_state(state):
    if "state" in state:
        return True
    return False

def search_plugin_and_apply(obj, plugin, func):
    returnobj = []
    keys = list(get_keys(obj, []))
    for i in range(len(keys)):
        if leaf_equals(keys[i], "plugin"):
            sub_obj = {}
            if len(keys[i]) == 1:
                sub_obj = obj
            else:
                sub_obj = get_value_from_key_array(obj, keys[i][1:])
            if sub_obj["plugin"] == plugin:
                returnobj.extend(func(sub_obj["state"]))
        elif leaf_equals(keys[i], "state"):
            state = get_value_from_key_array(obj, keys[i])
            if type(state) == list:
                for sub_obj in state:
                    if not sub_obj:
                        continue
                    returnobj.extend(search_plugin_and_apply(sub_obj, plugin, func))
    return returnobj

In [5]:
# Hier wird nur der Content geholt. Falls dieser ein Youtube-Video ist, könnte description und eventuell reasoning auch interessant sein.
def select_all_entities():
    return pd.read_sql("""
        SELECT entity.id, value
        FROM entity_revision_field, entity
        where field = "content"
            AND entity.current_revision_id = entity_revision_field.entity_revision_id;
        """, db)

In [4]:
def filter_non_trashed(revision):
    return not revision["trashed"]

def sortId(revision):
    return revision["id"]

def get_newest_unrevised_revision(id, bot):
    try:
        response = bot.api_call(query="""
            {
                uuid(id: %s) {
                    __typename
                    title
                    trashed
                    ... on Applet {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                url
                            }
                        }
                    }
                    ... on Article {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Course {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on CoursePage {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Event {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Exercise {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on ExerciseGroup {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                cohesive
                            }
                        }
                    }
                    ... on GroupedExercise {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Solution {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Video {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                url
                            }
                        }
                    }
                }
            }
        """ % id)
    except Exception as e:
        print(e)
        print("Some problem with entity id %s" % id)
        raise e
    revisions = response["uuid"]["revisions"]["nodes"]
    legal_revisions = list(filter(filter_non_trashed, revisions))
    legal_revisions.sort(reverse = True, key = sortId)
    return {
        "revision": legal_revisions[0],
        "metadata": {
            "type": response["uuid"]["__typename"],
            "title": response["uuid"]["title"],
            "trashed": response["uuid"]["trashed"]
        }
    }

def save_revision(id, new_revision, rules, metadata, bot):
    set_entity_input = """{
        changes: {rules},
        subscribeThis: True,
        subscribeThisByEmail: False,
        needsReview: True,
        entityId: id,
        title: metadata["metadata"]["title"],
        content: new_revision
    }"""
    match metadata["metadata"]["type"]:
        case "Applet":
            try:
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setApplet(input: {
                                changes: %s
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                title: %s,
                                content: %s,
                                url: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), metadata["metadata"]["title"], new_revision, metadata["revision"]["url"]))
            except Exception as e:
                print(id)
                raise e
        case "Article":
            query = """
                mutation {
                    entity {
                        setArticle(input: {
                            changes: "%s",
                            subscribeThis: true,
                            subscribeThisByEmail: false,
                            needsReview: true,
                            entityId: %s,
                            title: "%s"
                            content: "%s"
                        }) {
                            success
                        }
                    }
                }
            """ % (rules, str(id), metadata["metadata"]["title"], new_revision)
            try:
                response = bot.api_call(query)
            except Exception as e:
                print("id")
                print(id)
                raise e
        case "Course":
            try:
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setCourse(input: {
                                    changes: %s,
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    title: %s,
                                    content: %s
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), metadata["metadata"]["title"], new_revision))
            except Exception as e:
                print(id)
                raise e
        case "CoursePage":
            try:
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setCoursePage(input: {
                                    changes: '%s',
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    title: '%s',
                                    content: '%s'
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), metadata["metadata"]["title"], new_revision))
            except Exception as e:
                print(id)
                raise e
        case "Event":
            try:
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setEvent(input: {
                                    changes: %s,
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    title: %s,
                                    content: %s
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), metadata["metadata"]["title"], new_revision))
            except Exception as e:
                print(id)
                raise e
        case "Exercise":
            try:
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setExercise(input: {
                                changes: %s,
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                content: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), new_revision))
            except Exception as e:
                print(id)
                raise e
        case "ExerciseGroup":
            try:
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setExerciseGroup(input: {
                                    changes: %s,
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    content: %s
                                    cohesive: %s
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), new_revision, metadata["revision"]["cohesive"]))
            except Exception as e:
                print(id)
                raise e
        case "GroupedExercise":
            try:
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setGroupedExercise(input: {
                                changes: %s,
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                content: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), new_revision))
            except Exception as e:
                print(id)
                raise e
        case "Solution":
            try:
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setSolution(input: {
                                changes: %s,
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                content: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), new_revision))
            except Exception as e:
                print(id)
                raise e
        case "Video":
            try:
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setVideo(input: {
                                changes: %s
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                title: %s,
                                content: %s,
                                url: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), metadata["metadata"]["title"], new_revision, metadata["revision"]["url"]))
            except Exception as e:
                print(id)
                raise e
    print("saved")